# Capstone Project - The Battle of Neighborhoods

# Problem Statement

Lockdown is being lifted in London and it is time to check out museums in London again. However, we do not want to go to the places where there are too many Covid-19 cases. Where are the best museums in London that are also located in a borough with low Covid-19 cases?

# Data Description